* 점포 별 판매량의 추이를 그래프로 표현!
* 상품의 종류를 나누어 표현 !
* 가장 많이 팔린 점포만 확인!

In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('font', family='Malgun Gothic')

In [56]:
# 음료회사 dataset
df1 = pd.read_csv('Data09.csv',encoding='cp949')

In [58]:
df1.shape

(117, 646)

In [60]:
df1.head()

,순번,상품코드,상품명,합계,(1154)의정부점,(1155)별내점,(1156)풍산점,(1157)김포한강점,(1158)세종점,(1159)킨텍스점,...,(L21)위시티점,(L23)걸포동점,(L27)통일동산점,(L35)성대점,(L36)병점점,(L38)율전점,(L39)본오점,(L40)검암점,(L44)신길점,(L46)용호점
0,NaN,NaN,NaN,수량,수량,수량,수량,수량,수량,수량,...,수량,수량,수량,수량,수량,수량,수량,수량,수량,수량
1,NaN,NaN,합 계,"2,029,096","6,829","8,374","12,626","11,767","10,912","7,062",...,"2,392","1,291","4,506","2,364","2,739","1,782","3,129","1,712","3,189","2,347"
2,1.0,1801828.0,BW1 473ml캔(수입),"65,402",174,239,432,199,220,263,...,0,25,76,72,54,72,15,38,27,89
3,2.0,1801859.0,BW1 355ml (낱캔),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.0,40786179.0,RB 500ml,"2,158",4,9,10,27,0,11,...,0,0,0,0,0,0,0,0,0,0


In [62]:
# column 이름
df1.columns

Index(['순번', '상품코드', '상품명', ' 합계', '(1154)의정부점', '(1155)별내점', '(1156)풍산점',
       '(1157)김포한강점', '(1158)세종점', '(1159)킨텍스점',
       ...
       '(L21)위시티점', '(L23)걸포동점', '(L27)통일동산점', '(L35)성대점', '(L36)병점점',
       '(L38)율전점', '(L39)본오점', '(L40)검암점', '(L44)신길점', '(L46)용호점'],
      dtype='object', length=646)

### **Pandas stack**

In [64]:
# 순번과 상품코드를 없앤 데이터프레임 생성하기
stack_data = df1.drop(columns=['순번','상품코드']).set_index('상품명').stack()

In [66]:
df2 = pd.DataFrame(stack_data).reset_index()
df2

,상품명,level_1,0
0,NaN,합계,수량
1,NaN,(1154)의정부점,수량
2,NaN,(1155)별내점,수량
3,NaN,(1156)풍산점,수량
4,NaN,(1157)김포한강점,수량
...,...,...,...
75226,CAT1 1.6L,(L38)율전점,54
75227,CAT1 1.6L,(L39)본오점,28
75228,CAT1 1.6L,(L40)검암점,9
75229,CAT1 1.6L,(L44)신길점,28


### **Pandas Melt**

In [68]:
region_list = df1.columns.tolist()[4:]

In [70]:
pd.melt(df1, id_vars=['상품명'], value_vars=region_list)

,상품명,variable,value
0,NaN,(1154)의정부점,수량
1,합 계,(1154)의정부점,"6,829"
2,BW1 473ml캔(수입),(1154)의정부점,174
3,BW1 355ml (낱캔),(1154)의정부점,0
4,RB 500ml,(1154)의정부점,4
...,...,...,...
75109,CAT1 355ml*12캔+카카오 에코백+치,(L46)용호점,0
75110,CAT2 355*24캔 + 진라면 6캔,(L46)용호점,0
75111,CAT2 1.6L*6,(L46)용호점,0
75112,CAT2 1.6L,(L46)용호점,122


* 상품명 missing value 제거하기

In [72]:
# ~ : 상품명이 null이 아닌 값을 찾기
cond1 = df2['상품명'].isnull()
cond2 = df2['상품명'] != '합 계'
cond3 = df2['level_1'] != ' 합계'
df3 = df2.loc[(~cond1) & (cond2) & (cond3)]
df4 = df3.rename(columns={'level_1':'점포명',0:'판매량'})
df4

,상품명,점포명,판매량
1287,BW1 473ml캔(수입),(1154)의정부점,174
1288,BW1 473ml캔(수입),(1155)별내점,239
1289,BW1 473ml캔(수입),(1156)풍산점,432
1290,BW1 473ml캔(수입),(1157)김포한강점,199
1291,BW1 473ml캔(수입),(1158)세종점,220
...,...,...,...
75226,CAT1 1.6L,(L38)율전점,54
75227,CAT1 1.6L,(L39)본오점,28
75228,CAT1 1.6L,(L40)검암점,9
75229,CAT1 1.6L,(L44)신길점,28


* 제품별 분류하기 (label)

In [74]:
def func1(row):
    if 'CAT' in row:
        return 'C Type'
    elif 'STL' in row:
        return 'S Type'
    elif 'BW' in row:
        return 'B Type'
    else:
        return 'Non Type'

In [76]:
df4['제품군'] = df4['상품명'].apply(func1)

In [78]:
df4['제품군'].value_counts()

Non Type    44298
C Type      17334
B Type       6420
S Type       5778
Name: 제품군, dtype: int64

In [80]:
# non type이 아닌 제품군 확인
cond1 = df4['제품군'] != 'Non Type'
df5 = df4.loc[cond1]

In [ ]:
# 데이터 전처리 후 info까지 확인해야한다.
df5.info()

In [81]:
# 판매량이 object -> integer로 변환
df5['판매량'].astype(int)

ValueError: invalid literal for int() with base 10: '1,000'

In [83]:
'''
invalid literal for int() with base 10: '1,000
쉼표때문에 object type 생성 따라서 ,를 제거해야한다
''' 

def func2(row):
    if len(row) > 3:
        result = row.split(',')
        return result[0] + result[1]
    else:
        return row


df5['판매량(int)'] = df5['판매량'].apply(func2).astype(int)

C:\ananconda\envs\multi\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [84]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29532 entries, 1287 to 75230
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   상품명       29532 non-null  object
 1   점포명       29532 non-null  object
 2   판매량       29532 non-null  object
 3   제품군       29532 non-null  object
 4   판매량(int)  29532 non-null  int32 
dtypes: int32(1), object(4)
memory usage: 1.2+ MB


#### 가장 많이 팔린 점포 수를 pivot_table를 통해서 만든다.

In [85]:
df6 = pd.pivot_table(data=df5, index=['점포명','제품군'], 
               values='판매량(int)',aggfunc='sum').reset_index()
df6

,점포명,제품군,판매량(int)
0,(1154)의정부점,B Type,1236
1,(1154)의정부점,C Type,3119
2,(1154)의정부점,S Type,434
3,(1155)별내점,B Type,628
4,(1155)별내점,C Type,4438
...,...,...,...
1921,(L44)신길점,C Type,2473
1922,(L44)신길점,S Type,52
1923,(L46)용호점,B Type,236
1924,(L46)용호점,C Type,1632


In [86]:
# 판매량에 대한 내림차순으로 정렬
df7 = df6.sort_values(by='판매량(int)',ascending=False).head(200)
df7.to_csv('result_0626.csv',encoding='cp949')
df7

,점포명,제품군,판매량(int)
1393,(610)부천점,C Type,13397
1387,(600)안양점,C Type,11806
1594,(966)성남점,C Type,11732
1372,(550)은평점,C Type,10278
1390,(601)시화점,C Type,9480
...,...,...,...
1588,(963)수색점,C Type,2155
706,(2674)동탄호수점,C Type,2151
1355,(470)성수점,S Type,2147
1356,(480)월배점,B Type,2140


# python Dash / Plotly  

* Python -> Wep : Django / Flask
* react.js / flask -> Dash / plotly (데이터 시각화를 어플리케이션 하는데 특화되어 있다.)
    - Data 시각화 App -> 간단한 형태의 데이터 시각화를 만들 수 있다. 

In [87]:
import plotly.express as px

In [89]:
fig = px.bar(df7, x='점포명', y='판매량(int)',color='제품군')
fig.show()